In [119]:
import itertools
import os
from collections import Counter

import numpy as np
import pandas as pd
from IPython.core.display import display
print(os.listdir("../input"))

['.DS_Store', 'imbd-sf']


In [120]:
data = pd.read_csv('../input/imbd-sf/data.csv')
data.head(5)

,imdb_id,popularity,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year
0,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015
1,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015
2,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015
3,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015
4,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015


In [121]:
len(data)

1890

# Предобработка датасета

In [122]:
# сюда можем вписать создание новых колонок в датасете
answer_ls = [] # создадим список с ответами. сюда будем добавлять ответы по мере прохождения теста

# Добавим нужных нам показателей
def has_genre(record, genre):
    return record['genres'].str.contains(genre)

def get_release_month(record):
    return record['release_date'].str.split('/').str[0].astype(int);

data = data.assign(
    # profit  - прибыль на основе бюджета и дохода
    profit = lambda x: x['revenue'] - x['budget'],
    # is_action       - показатель того, имеет ли фильм 'Action' в списке жанров
    is_action = lambda x: has_genre(x, 'Action'),
    # is_adventure    - показатель того, имеет ли фильм 'Adventure' в списке жанров
    is_adventure = lambda x: has_genre(x, 'Adventure'),
    # is_drama        - показатель того, имеет ли фильм 'Drama' в списке жанров
    is_drama = lambda x: has_genre(x, 'Drama'),
    # is_comedy       - показатель того, имеет ли фильм 'Comedy' в списке жанров
    is_comedy = lambda x: has_genre(x, 'Comedy'),
    # is_thriller     - показатель того, имеет ли фильм 'Thriller' в списке жанров
    is_thriller = lambda x: has_genre(x, 'Thriller'),
    # is_thriller     - месяц выпуска фильма
    release_month = lambda x: get_release_month(x),
)
display(data.head(5))

# Создадим справочник с информацией о том, в каких фильмах играли актеры
cast = pd.DataFrame(data.cast.str.split('|').tolist(), index = data.imdb_id).stack()
cast = cast.reset_index([0, 'imdb_id'])
cast.columns = ['imdb_id', 'actor']
display(cast.head(5))

# Создадим справочник с информацией о том, какие студии были задействованы при съемках каждого фильма
production_companies = pd.DataFrame(data.production_companies.str.split('|').tolist(), index = data.imdb_id).stack()
production_companies = production_companies.reset_index([0, 'imdb_id'])
production_companies.columns = ['imdb_id', 'company']
display(production_companies.head(5))

# Создадим справочник с информацией о том, какой режиссер работал над каждым фильмом
film_directors = pd.DataFrame(data.director.str.split('|').tolist(), index = data.imdb_id).stack()
film_directors = film_directors.reset_index([0, 'imdb_id'])
film_directors.columns = ['imdb_id', 'film_director']
display(film_directors.head(5))

,imdb_id,popularity,budget,revenue,original_title,cast,director,tagline,overview,runtime,...,vote_count,vote_average,release_year,profit,is_action,is_adventure,is_drama,is_comedy,is_thriller,release_month
0,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,...,5562,6.5,2015,1363528810,True,True,False,False,True,6
1,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,...,6185,7.1,2015,228436354,True,True,False,False,True,5
2,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,...,2480,6.3,2015,185238201,False,True,False,False,True,3
3,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,...,5292,7.5,2015,1868178225,True,True,False,False,False,12
4,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,...,2947,7.3,2015,1316249360,True,False,False,False,True,4


,imdb_id,actor
0,tt0369610,Chris Pratt
1,tt0369610,Bryce Dallas Howard
2,tt0369610,Irrfan Khan
3,tt0369610,Vincent D'Onofrio
4,tt0369610,Nick Robinson


,imdb_id,company
0,tt0369610,Universal Studios
1,tt0369610,Amblin Entertainment
2,tt0369610,Legendary Pictures
3,tt0369610,Fuji Television Network
4,tt0369610,Dentsu


,imdb_id,film_director
0,tt0369610,Colin Trevorrow
1,tt1392190,George Miller
2,tt2908446,Robert Schwentke
3,tt2488496,J.J. Abrams
4,tt2820852,James Wan


# 1. У какого фильма из списка самый большой бюджет?

Варианты ответов:
1. The Dark Knight Rises (tt1345836)
2. Spider-Man 3 (tt0413300)
3. Avengers: Age of Ultron (tt2395427)
4. The Warrior's Way	(tt1032751)
5. Pirates of the Caribbean: On Stranger Tides (tt1298650)

In [123]:
solution = data[data.budget == data.budget.max()]
answer_ls.append("4. The Warrior's Way	(tt1032751)")

# 2. Какой из фильмов самый длительный (в минутах)

Варианты ответов:
1. The Lord of the Rings: The Return of the King	(tt0167260)
2. Gods and Generals	(tt0279111)
3. King Kong	(tt0360717)
4. Pearl Harbor	(tt0213149)
5. Alexander	(tt0346491)

In [124]:
solution = data[data.runtime == data.runtime.max()]
answer_ls.append("2. Gods and Generals	(tt0279111)")

# 3. Какой из фильмов самый короткий (в минутах)

Варианты ответов:
1. Home on the Range	tt0299172
2. The Jungle Book 2	tt0283426
3. Winnie the Pooh	tt1449283
4. Corpse Bride	tt0121164
5. Hoodwinked!	tt0443536

In [125]:
solution = data[data.runtime == data.runtime.min()]
answer_ls.append("3. Winnie the Pooh	tt1449283")

# 4. Средняя длительность фильма?

Варианты ответов:
1. 115
2. 110
3. 105
4. 120
5. 100

In [126]:
solution = data.runtime.mean()
answer_ls.append("2. 110")

# 5. Средняя длительность фильма по медиане?

Варианты ответов:
1. 106
2. 112
3. 101
4. 120
5. 115

In [127]:
solution = data.runtime.median()
answer_ls.append("2. 106")

# 6. Какой самый прибыльный фильм?

Варианты ответов:
1. The Avengers	tt0848228
2. Minions	tt2293640
3. Star Wars: The Force Awakens	tt2488496
4. Furious 7	tt2820852
5. Avatar	tt0499549

In [128]:
solution = data[data.profit == data.profit.max()]
answer_ls.append("5. Avatar	tt0499549")

# 7. Какой фильм самый убыточный?

Варианты ответов:
1. Supernova tt0134983
2. The Warrior's Way tt1032751
3. Flushed Away	tt0424095
4. The Adventures of Pluto Nash	tt0180052
5. The Lone Ranger	tt1210819

In [129]:
solution = data[data.profit == data.profit.min()]
answer_ls.append("2. The Warrior's Way tt1032751")

# 8. Сколько всего фильмов в прибыли?

Варианты ответов:
1. 1478
2. 1520
3. 1241
4. 1135
5. 1398

In [130]:
solution = len(data[data.profit > 0])
answer_ls.append("1. 1478")

# 9. Самый прибыльный фильм в 2008 году?

Варианты ответов:
1. Madagascar: Escape 2 Africa	tt0479952
2. Iron Man	tt0371746
3. Kung Fu Panda	tt0441773
4. The Dark Knight	tt0468569
5. Mamma Mia!	tt0795421

In [131]:
profits_2008 = data[data.release_year == 2008]

solution = profits_2008[profits_2008.profit == profits_2008.profit.max()]
answer_ls.append("4. The Dark Knight	tt0468569")

# 10. Самый убыточный фильм за период с 2012 по 2014 (включительно)?

Варианты ответов:
1. Winter's Tale	tt1837709
2. Stolen	tt1656186
3. Broken City	tt1235522
4. Upside Down	tt1374992
5. The Lone Ranger	tt1210819

In [132]:
profits_2012_2014 = data[(data.release_year >= 2012) & (data.release_year <= 2014)]

solution = profits_2012_2014[profits_2012_2014.profit == profits_2012_2014.profit.min()]
answer_ls.append("5. The Lone Ranger	tt1210819")

# 11. Какого жанра фильмов больше всего?

Варианты ответов:
1. Action
2. Adventure
3. Drama
4. Comedy
5. Thriller

In [133]:
data.describe(include=['bool'])
answer_ls.append('3. Drama')

# 12. Какого жанра среди прибыльных фильмов больше всего?

Варианты ответов:
1. Drama
2. Comedy
3. Action
4. Thriller
5. Adventure

In [134]:
data[data.profit > 0].describe(include=['bool'])
answer_ls.append('1. Drama')

# 13. Кто из режиссеров снял больше всего фильмов?

Варианты ответов:
1. Steven Spielberg
2. Ridley Scott
3. Steven Soderbergh
4. Christopher Nolan
5. Clint Eastwood

In [135]:
solution = data.director.value_counts()
answer_ls.append('3. Steven Soderbergh')

# 14. Кто из режиссеров снял больше всего Прибыльных фильмов?

Варианты ответов:
1. Steven Soderbergh
2. Clint Eastwood
3. Steven Spielberg
4. Ridley Scott
5. Christopher Nolan

In [136]:
solution = data[data.profit > 0].director.value_counts()
answer_ls.append('4. Ridley Scott')

# 15. Кто из режиссеров принес больше всего прибыли?

Варианты ответов:
1. Steven Spielberg
2. Christopher Nolan
3. David Yates
4. James Cameron
5. Peter Jackson

In [137]:
solution = data.groupby('director').profit.sum().sort_values(ascending=False)
answer_ls.append('5. Peter Jackson')

# 16. Какой актер принес больше всего прибыли?

Варианты ответов:
1. Emma Watson
2. Johnny Depp
3. Michelle Rodriguez
4. Orlando Bloom
5. Rupert Grint

In [138]:
joined = data.merge(cast, on='imdb_id', how='inner')

solution = joined.groupby('actor').profit.sum().sort_values(ascending=False)
answer_ls.append('1. Emma Watson')

# 17. Какой актер принес меньше всего прибыли в 2012 году?

Варианты ответов:
1. Nicolas Cage
2. Danny Huston
3. Kirsten Dunst
4. Jim Sturgess
5. Sami Gayle

In [139]:
profits_2012 = data[data.release_year == 2012]
joined = profits_2012.merge(cast, on='imdb_id', how='inner')

solution = joined.groupby('actor').profit.sum().sort_values(ascending=True)
answer_ls.append('3. Kirsten Dunst')

# 18. Какой актер снялся в большем количестве высокобюджетных фильмов? (в фильмах где бюджет выше среднего по данной выборке)

Варианты ответов:
1. Tom Cruise
2. Mark Wahlberg
3. Matt Damon
4. Angelina Jolie
5. Adam Sandler

In [140]:
high_budget = data[data.budget > data.budget.mean()]
joined = high_budget.merge(cast, on='imdb_id', how='inner')

solution = joined.actor.value_counts()
answer_ls.append('3. Matt Damon')

# 19. В фильмах какого жанра больше всего снимался Nicolas Cage?

Варианты ответа:
1. Drama
2. Action
3. Thriller
4. Adventure
5. Crime

In [141]:
caged_films = cast[cast.actor == 'Nicolas Cage']

solution = data.merge(caged_films, on='imdb_id', how='right').describe(include=['bool'])
answer_ls.append('2. Action')

# 20. Какая студия сняла больше всего фильмов?

Варианты ответа:
1. Universal Pictures (Universal)
2. Paramount Pictures
3. Columbia Pictures
4. Warner Bros
5. Twentieth Century Fox Film Corporation

In [142]:
joined = data.merge(production_companies, on='imdb_id', how='inner')

solution = joined.company.value_counts()
answer_ls.append('1. Universal Pictures (Universal)')

# 21. Какая студия сняла больше всего фильмов в 2015 году?

Варианты ответа:
1. Universal Pictures
2. Paramount Pictures
3. Columbia Pictures
4. Warner Bros
5. Twentieth Century Fox Film Corporation

In [143]:
films_2015 = data[data.release_year == 2015]
joined = films_2015.merge(production_companies, on='imdb_id', how='inner')

solution = joined.company.value_counts()
answer_ls.append('4. Warner Bros')

# 22. Какая студия заработала больше всего денег в жанре комедий за все время?

Варианты ответа:
1. Warner Bros
2. Universal Pictures (Universal)
3. Columbia Pictures
4. Paramount Pictures
5. Walt Disney

In [144]:
comedies = data[data.is_comedy == True]
joined = comedies.merge(production_companies, on='imdb_id', how='inner')

solution = joined.groupby('company').profit.sum().sort_values(ascending=False)
answer_ls.append('2. Universal Pictures (Universal)')

# 23. Какая студия заработала больше всего денег в 2012 году?

Варианты ответа:
1. Universal Pictures (Universal)
2. Warner Bros
3. Columbia Pictures
4. Paramount Pictures
5. Lucasfilm

In [145]:
films_2012 = data[data.release_year == 2012]
joined = films_2012.merge(production_companies, on='imdb_id', how='inner')

solution = joined.groupby('company').profit.sum().sort_values(ascending=False)
answer_ls.append('3. Columbia Pictures')

# 24. Самый убыточный фильм от Paramount Pictures

Варианты ответа:

1. K-19: The Widowmaker tt0267626
2. Next tt0435705
3. Twisted tt0315297
4. The Love Guru tt0811138
5. The Fighter tt0964517

In [146]:
paramount_films = production_companies[production_companies.company == 'Paramount Pictures']
joined = data.merge(paramount_films, on='imdb_id', how='inner')

solution = joined[joined.profit == joined.profit.min()]
answer_ls.append('1. K-19: The Widowmaker tt0267626')

# 25. Какой Самый прибыльный год (заработали больше всего)?

Варианты ответа:
1. 2014
2. 2008
3. 2012
4. 2002
5. 2015

In [147]:
solution = data.groupby('release_year').profit.sum().sort_values(ascending=False)
answer_ls.append('5. 2015')

# 26. Какой Самый прибыльный год для студии Warner Bros?

Варианты ответа:
1. 2014
2. 2008
3. 2012
4. 2010
5. 2015

In [148]:
warner_films = production_companies[production_companies.company.str.contains('Warner Bros')]
joined = data.merge(warner_films, on='imdb_id', how='inner')

solution = joined.groupby('release_year').profit.sum().sort_values(ascending=False)
answer_ls.append('1. 2014')

# 27. В каком месяце за все годы суммарно вышло больше всего фильмов?

Варианты ответа:
1. Январь
2. Июнь
3. Декабрь
4. Сентябрь
5. Май

In [149]:
solution = data.release_month.value_counts()
answer_ls.append('4. Сентябрь')

# 28. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

Варианты ответа:
1. 345
2. 450
3. 478
4. 523
5. 381

In [150]:
solution = len(data[data.release_month.isin([6,7,8])])
answer_ls.append('2. 450')

# 29. Какой режиссёр выпускает (суммарно по годам) больше всего фильмов зимой?

Варианты ответов:
1. Steven Soderbergh
2. Christopher Nolan
3. Clint Eastwood
4. Ridley Scott
5. Peter Jackson

In [151]:
winter_films = data[data.release_month.isin([1,2,12])]

solution = winter_films.director.value_counts()
answer_ls.append('5. Peter Jackson')

# 30. Какой месяц чаще всего по годам самый прибыльный?

Варианты ответа:
1. Январь
2. Июнь
3. Декабрь
4. Сентябрь
5. Май

In [152]:
pivot = data.pivot_table(values=['profit'], index=['release_month'], columns=['release_year'], aggfunc='sum')

solution = pivot.idxmax().value_counts()
answer_ls.append('2. Июнь')

# 31. Названия фильмов какой студии в среднем самые длинные по количеству символов?

Варианты ответа:
1. Universal Pictures (Universal)
2. Warner Bros
3. Jim Henson Company, The
4. Paramount Pictures
5. Four By Two Productions

In [153]:
joined = data.merge(production_companies, on='imdb_id', how='inner')

solution = joined.groupby(['company'])['original_title'].apply(lambda x: np.mean(x.str.len())).sort_values(ascending=False)
answer_ls.append('5. Four By Two Productions')

# 32. Названия фильмов какой студии в среднем самые длинные по количеству слов?

Варианты ответа:
1. Universal Pictures (Universal)
2. Warner Bros
3. Jim Henson Company, The
4. Paramount Pictures
5. Four By Two Productions

In [154]:
joined = data.merge(production_companies, on='imdb_id', how='inner')

display(joined[joined.company == 'Four By Two Productions'])

solution = joined.groupby(['company'])['original_title'].apply(lambda x: np.mean(x.str.split().str.len())).sort_values(ascending=False)
display(solution)
answer_ls.append('5. Four By Two Productions')

,imdb_id,popularity,budget,revenue,original_title,cast,director,tagline,overview,runtime,...,vote_average,release_year,profit,is_action,is_adventure,is_drama,is_comedy,is_thriller,release_month,company
4688,tt0443453,1.930735,18000000,261572744,Borat: Cultural Learnings of America for Make ...,Sacha Baron Cohen|Ken Davitian|Luenell|Pamela ...,Larry Charles,"Come to Kazakhstan, it's nice!",Kazakh journalist Borat Sagdiyev travels to Am...,82,...,6.4,2006,243572744,False,False,False,True,False,11,Four By Two Productions


company
Four By Two Productions                  12.0
Jim Henson Company, The                  10.0
Polsky Films                              9.0
The Saul Zaentz Company                   9.0
Dos Corazones                             9.0
                                         ... 
Many Rivers Productions                   1.0
Maple Shade Films                         1.0
Blue Productions                          1.0
Blue Sea Productions                      1.0
Great Wight Productions/ Osford Films     1.0
Name: original_title, Length: 1772, dtype: float64

# 33. Сколько разных слов используется в названиях фильмов?(без учета регистра)

Варианты ответа:
1. 6540
2. 1002
3. 2461
4. 28304
5. 3432

In [155]:
words = data.original_title.str.lower().str.split().tolist();
flat_words = list(itertools.chain.from_iterable(words))

solution = len(Counter(flat_words).keys())
answer_ls.append('3. 2461')

# 34. Какие фильмы входят в 1 процент лучших по рейтингу?

Варианты ответа:
1. Inside Out, Gone Girl, 12 Years a Slave
2. BloodRayne, The Adventures of Rocky and Bullwinkle
3. The Lord of the Rings: The Return of the King, Upside Down
4. 300, Lucky Number Slevin
5. Upside Down, 300, Inside Out, The Lord of the Rings: The Return of the King

In [156]:
solution = data[data.vote_average > data.vote_average.quantile(0.99)]['original_title']
answer_ls.append("1. Inside Out, Gone Girl, 12 Years a Slave")

# 35. Какие актеры чаще всего снимаются в одном фильме вместе

Варианты ответа:
1. Johnny Depp & Helena Bonham Carter
2. Hugh Jackman & Ian McKellen
3. Vin Diesel & Paul Walker
4. Adam Sandler & Kevin James
5. Daniel Radcliffe & Rupert Grint

In [157]:
pairs = [
    ["Johnny Depp", "Helena Bonham Carter"],
    ["Hugh Jackman", "Ian McKellen"],
    ["Vin Diesel", "Paul Walker"],
    ["Adam Sandler", "Kevin James"],
    ["Daniel Radcliffe", "Rupert Grint"]
]

for pair in pairs:
    films = data[(data.cast.str.contains(pair[0])) & (data.cast.str.contains(pair[1]))]
    print(pair, len(films))

answer_ls.append("5. Daniel Radcliffe & Rupert Grint")

['Johnny Depp', 'Helena Bonham Carter'] 6
['Hugh Jackman', 'Ian McKellen'] 5
['Vin Diesel', 'Paul Walker'] 5
['Adam Sandler', 'Kevin James'] 5
['Daniel Radcliffe', 'Rupert Grint'] 8


# 36. У какого из режиссеров выше вероятность выпустить фильм в прибыли? (5 баллов)101

Варианты ответа:
1. Quentin Tarantino
2. Steven Soderbergh
3. Robert Rodriguez
4. Christopher Nolan
5. Clint Eastwood

In [158]:
wanted_directors = ['Quentin Tarantino','Steven Soderbergh','Robert Rodriguez','Christopher Nolan','Clint Eastwood']

joined = data.merge(film_directors, on='imdb_id', how='inner')
grouped = joined.groupby('film_director')
aggregated = grouped.profit.apply(lambda x: pd.Series([(x).count(), (x > 0).sum() / (x).count()])).unstack().reset_index()

solution = aggregated[aggregated.film_director.isin(wanted_directors)]
answer_ls.append('4. Christopher Nolan')

# Submission

In [159]:
len(answer_ls)

36

In [160]:
pd.DataFrame({'Id':range(1,len(answer_ls)+1), 'Answer':answer_ls}, columns=['Id', 'Answer'])

,Id,Answer
0,1,4. The Warrior's Way\t(tt1032751)
1,2,2. Gods and Generals\t(tt0279111)
2,3,3. Winnie the Pooh\ttt1449283
3,4,2. 110
4,5,2. 106
5,6,5. Avatar\ttt0499549
6,7,2. The Warrior's Way tt1032751
7,8,1. 1478
8,9,4. The Dark Knight\ttt0468569
9,10,5. The Lone Ranger\ttt1210819
